# Assignment with Rubric

## Authors

R. Treharne

## Overview

This notebook will allow a teacher to extract rubric scores from submissions.

## Requirements

If you are running this in Google Colab then you will need to click the "Copy to Drive" button before you can interact with the notebook. Also, you will need to install the `canvasapi` Python module by running the cell below:

In [ ]:
# If using Colab, run this cell
!pip install canvasapi

## Usage

Watch the following video to see how to use the notebook:

<a href="https://youtu.be/GRSc9Bir6dE">
    <img src="./thumb.png" alt="YouTube Thumbnail" width="200"/>
</a>

Run the following cells to generate a report on an assignment. 

You will be prompted to enter the following:

+ Your Canvas URL (e.g. "https://canvas.instructure.com")
+ Your Canvas API Token
+ Your Course ID
+ Your Assignment ID

In [ ]:
# Important. Run this cell and respond to the prompts to set up your Canvas API object.

import getpass
import pandas as pd
from canvasapi import Canvas
import datetime

CANVAS_URL = input("Input your canvas URL: ")
CANVAS_TOKEN = getpass.getpass("Input your canvas API TOKEN: ")

canvas = Canvas(CANVAS_URL, CANVAS_TOKEN)

In [ ]:
# Important. Run this cell.

COURSE_ID = input("Input the canvas course ID: ")
ASSIGNMENT_ID = input("Input the assignment ID: ")

course = canvas.get_course(COURSE_ID)
assignment = course.get_assignment(ASSIGNMENT_ID)

In [ ]:
# Important. Run this cell.
# This cell contains the functions that will be used to generate the report.
# Do not edit this .

def get_enrollments(course):
    enrollments = [x for x in course.get_enrollments(include=["user"])]
    return enrollments

def get_user_from_id(user_id, enrollments):
    for enrollment in enrollments:
        if enrollment.user_id == user_id:
            return enrollment.user["sortable_name"]
        
def get_rubric_categories(rubric):
    rubric_categories = [x["description"] for x in rubric.data]
    return rubric_categories

def generate_report(course, assignment):

    print("Getting enrollments...")
    enrollments = get_enrollments(course)
    
    print("Getting submissions...")
    submissions = [x for x in assignment.get_submissions(include=["submission_comments", "user", "rubric_assessment"])]

    rubric = course.get_rubric(assignment.rubric_settings["id"])

    rubric_categories = get_rubric_categories(rubric)

    report = []

    "Generating report..."
    for submission in submissions:
        
        row = {}

        try:
            marker_name = get_user_from_id(submission.grader_id, enrollments)
        except:
            marker_name = ""

        row["user_id"] = submission.user_id
        row["sis_user_id"] = submission.user["sis_user_id"]
        row["name"] = submission.user["sortable_name"]
        row["submitted_at"] = submission.submitted_at
        row["late_sec"] = submission.seconds_late
        row["workflow_state"] = submission.workflow_state
        row["graded_at"] = submission.graded_at
        row["marker_name"] = marker_name
        row["score"] = submission.score

        try:
            row["comments"] = submission.submission_comments
        except:
            row["comments"] = ""
        
        try:
            rubric_assessment = submission.rubric_assessment
        except:
            rubric_assessment = None

        if rubric_assessment:
            for cat, rat in zip(rubric_categories, rubric_assessment):
                try:
                    row["rubric_" + cat.lower().replace(" ", "_")] = rubric_assessment[rat]["points"]
                except:
                    row["rubric_" + cat.lower().replace(" ", "_")] = ""

        report.append(row)
    
    report = pd.DataFrame(report)

    # Get timestamp string
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")

    # Save report
    print("Saving report...")
    fname = f"{timestamp}_{str(COURSE_ID)}_{str(ASSIGNMENT_ID)}.csv"
    report.to_csv(fname, index=False)
    print(f"Report saved as {fname}")
    
    return report

In [ ]:
# Run this cell to generate the report and save it.
report = generate_report(course, assignment)

If you're using Google colab then click the "files" icon in the left toolbar and then download the file.